In [28]:
import numpy as np
%run functions.ipynb

In [3]:
from numpy.linalg import svd
class PVD():
    def __init__(self, data):
        self.observations=data
        self.left_loading=None
        self.right_loading=None
        self.core=None
        self.residual=None
    def fit(self):
        T=self.observations.shape[2]
        k=min(self.observations.shape[0],self.observations.shape[1])
        P=[]
        Q=[]
        for t in range(T):
            matrix=self.observations[:,:,t]
            U,Sigma,D=svd(matrix)
            #k=select_r(Sigma)
            #print(k)
            P.append((U[:,:k]@np.diag(Sigma[:k])))
            #print(P[-1].shape)
            Q.append((D.T[:,:k]@np.diag(Sigma[:k])))
            
        #P=np.array(P)
        #Q=np.array(Q)
        #print(P.shape,Q.shape)

        P=np.hstack(P)
        Q=np.hstack(Q)
        #print(P.shape,Q.shape)
        def eigen_decomp(M):
            eigenValues, eigenVectors = linalg.eig(M)

            idx = eigenValues.argsort()[::-1]   
            eigenValues = eigenValues[idx]
            eigenVectors = eigenVectors[:,idx]
            return np.real(eigenValues),np.real(eigenVectors)
        
        
        #L,_,__=svd(P)
        #print(_)
        evalue,L=eigen_decomp(P@P.T)
        r1=select_r(evalue)
        #R,_,__=svd(Q)
        #print(_)
        evalue,R=eigen_decomp(Q@Q.T)
        r2=select_r(evalue)
        self.left_loading= L[:,:r1]
        self.right_loading= R[:,:r2]
        self.core=np.einsum('ik,klt,lj->ijt',self.left_loading.T,self.observations,self.right_loading)
        self.residual=self.observations-np.einsum('ik,klt,lj->ijt',self.left_loading,self.core,self.right_loading.T)